In [14]:
!pip install underthesea
import pandas as pd
import re
from nltk import ngrams
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from underthesea import word_tokenize, text_normalize
import joblib
from sklearn.model_selection import train_test_split
import os
from sklearn.model_selection import cross_val_score


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [15]:
df = pd.read_csv('./input/dataset.csv')

In [16]:
columns = ['Text', 'label']
df.columns = columns
df

,Text,label
0,Đoạn cuối 2 người bên nhau thêm nhạc nền nữa m...,1
1,"Tiếc thật, đến tận bây giờ mới biết bộ phim nà...",1
2,Cười muốn chết luôn á,0
3,Hài thật rồi chưa bao giờ thấy con rối buồn cư...,0
4,2/9/2022 phim hay chỉ tiết cập diễn phụ. Rất c...,3
...,...,...
394,máy cái hình kinh quá ọe,2
395,nói chung là phim k hay. Nhạt,4
396,Kịch bản dở tệ,4
397,phim nhảm lắm,4


In [17]:
def standardize_data(row):
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)

    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ") \
        .replace(")", " ").replace("=", " ") \
        .replace("(", " ").replace("}", " ") 
    row = row.strip()
    return row

In [18]:
def tokenizer(row):
    return word_tokenize(row, format="text")

# sentence = ‘Chàng trai 9X Quảng Trị khởi nghiệp từ nấm sò’
# >>> word_tokenize(sentence, format=”text”)’Chàng_trai 9X Quảng_Trị khởi_nghiệp từ nấm sò’“`
# Work tokenizer chuyển 1 câu thành các word có nghĩa ( bao gồm cả từ đơn và từ ghép )

In [19]:
def normalize(row):
    return text_normalize(row) 
# Chuan hoa tu
# >>> text_normalize('Ðảm baỏ chất lựơng phòng thí nghịêm hoá học')
# 'Đảm bảo chất lượng phòng thí nghiệm hóa học'

In [20]:
def embedding(X_train, X_test):
    global  emb
    emb = TfidfVectorizer()
    emb.fit(X_train)
    X_train = emb.transform(X_train)
    X_test = emb.transform(X_test)

    # Save pkl file
    joblib.dump(emb, './input/tfidf.pkl')
    return X_train, X_test


In [21]:
df['label'].value_counts()

3    109
4     89
1     86
0     65
2     50
Name: label, dtype: int64

In [22]:
df['Text'] = df['Text'].str.lower()
df['Text'] = df['Text'].apply(standardize_data)
df['Text'] = df["Text"] = df.Text.apply(tokenizer)
df['Text'] = df["Text"] = df.Text.apply(normalize)

print(df)

with open("./input/stopwords.txt") as f:
    stopwords = f.read().splitlines() 

df['Text'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))



print(df)
df.head()

                                                  Text  label
0    đoạn cuối 2 người bên nhau thêm nhạc nền nữa m...      1
1    tiếc thật đến tận bây_giờ mới biết bộ phim này...      1
2                                cười muốn chết luôn á      0
3    hài thật rồi chưa bao_giờ thấy con_rối buồn_cư...      0
4    2/9/2022 phim hay chỉ tiết cập diễn phụ rất cả...      3
..                                                 ...    ...
394                           máy cái hình kinh quá ọe      2
395                       nói_chung là phim k hay nhạt      4
396                                     kịch_bản dở_tệ      4
397                                      phim nhảm lắm      4
398                              phim dỡ dã_man luôn á      4

[399 rows x 2 columns]
                                                  Text  label
0    đoạn 2 nhạc mê xem cảm_nhận niềm hạnh_phúc trả...      1
1            tiếc tận bộ phim phim hay cảm_xúc đã khóc      1
2                                            c

,Text,label
0,đoạn 2 nhạc mê xem cảm_nhận niềm hạnh_phúc trả...,1
1,tiếc tận bộ phim phim hay cảm_xúc đã khóc,1
2,cười chết,0
3,hài chưa con_rối buồn_cười lắm,0
4,2/9/2022 phim hay tiết cập diễn phụ cảm_động,3


In [23]:
X = df["Text"]
y = df["label"]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [25]:
X_train, X_test = embedding(X_train, X_test)

In [26]:
model = svm.SVC(kernel='linear', C = 1)
model.fit(X_train,y_train)


SVC(C=1, kernel='linear')

In [27]:
import pickle
with open('./input/model.pkl','wb') as f:
    pickle.dump(model,f)

In [28]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.75      0.78        12
           1       0.94      0.71      0.81        21
           2       0.71      0.62      0.67         8
           3       0.58      0.82      0.68        22
           4       0.80      0.71      0.75        17

    accuracy                           0.74        80
   macro avg       0.77      0.72      0.74        80
weighted avg       0.77      0.74      0.74        80



In [29]:
# df_test = pd.DataFrame(["phim xem xong mà muốn khóc"])
# df_test[0] = df_test[0].apply(standardize_data)
# df_test[0] = df_test[0].apply(tokenizer)
# X_val = df_test[0]
# emb = joblib.load('tfidf.pkl')
# X_val = emb.transform(X_val)
# result = model.predict(X_val)

# print(result[0])
# x = model.predict("Chiết khấu tương đối tốt. Giao hàng tương đối nhanh. Phục vụ tốt. ")